In [ ]:
#!python -m venv myenv
#!source myenv/bin/activate

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import minmax_scale

In [ ]:
# prompt: Read sheet Consolidado from this spreadsheet: /content/drive/Shareddrives/Moura_TAD/Moura x UPE Compartilhamento/Estudo Engenharia de Processos/GM60AD - Tensões Elemento invertido.xlsm

#df = pd.read_excel('GM60AD - Tensões Elemento invertido.xlsm', sheet_name='Consolidado')
df_full = pd.read_csv("C:/Users/carlo/OneDrive/GitHub/DS-Moura/UN10_LIA02_TAD02_database_completa.csv")
df_corrente = pd.read_csv("C:/Users/carlo/OneDrive/GitHub/DS-Moura/df_corrente_maio_2024.csv")
df_tensao = pd.read_csv("C:/Users/carlo/OneDrive/GitHub/DS-Moura/df_tensao_maio_2024.csv")
df_condutancia = pd.read_csv("C:/Users/carlo/OneDrive/GitHub/DS-Moura/df_admitancia_maio_2024.csv")
df_full = df_full.drop(columns='Unnamed: 0')
df_corrente = df_corrente.drop(columns='Unnamed: 0')
df_tensao = df_tensao.drop(columns='Unnamed: 0')
df_condutancia = df_condutancia.drop(columns='Unnamed: 0')
df_full['condutancia'] = df_full.corrente/df_full.tensao

In [ ]:
# Normalize voltage and current and calculate % of initial voltage
df_full['NormV'] = minmax_scale(df_full['tensao'],axis=0)

In [ ]:
df_full['NormC'] = minmax_scale(df_full['corrente'],axis=0)

In [ ]:
df_full['%_Vi'] = df_full['tensao']/df_full['tensao'].max()

In [ ]:
# removing the last 50 test value of the dataframe
df_full = df_full[ df_full['sequencia'] <= 250 ]
df_corrente = df_corrente.drop(columns=[str(i) for i in range(251, 301)])
df_tensao = df_tensao.drop(columns=[str(i) for i in range(251, 301)])
df_condutancia = df_condutancia.drop(columns=[str(i) for i in range(251, 301)])

In [ ]:
#create a new df for automl and normalizing them
df_tensaoin = df_tensao.copy()
df_correntein = df_corrente.copy()
df_condutanciain = df_condutancia.copy()

df_tensaoin.iloc[:,:251] = (df_tensao.iloc[:,:251]-df_full['tensao'].min())/(df_full['tensao'].max()-df_full['tensao'].min())
df_correntein.iloc[:,:251] = (df_corrente.iloc[:,:251]-df_full['corrente'].min())/(df_full['corrente'].max()-df_full['corrente'].min())
#df_condutanciain.iloc[:,5:306] = minmax_scale(df_condutancia.iloc[:,5:306],axis=0)

In [ ]:
class_mapping = { 0:'Erro desconhecido',
                 1:'Aprovado',
                 2:'Falha polaridade',
                 3:'Falha tensão CA baixa',
                 4:'Falha tensão CA alta',
                 5:'Falha tensão CF baixa',
                 6:'Falha tensão CF alta',
                 7:'Falha diferencial tensão',
                 8:'Falha delta corrente mínima',
                 9:'Falha delta corrente máxima'
}

# Create the 'class' column
df_full['class'] = df_full['valor'].map(class_mapping)
df_corrente['class'] = df_corrente['valor'].map(class_mapping)
df_correntein['class'] = df_corrente['valor'].map(class_mapping)
df_tensao['class'] = df_tensao['valor'].map(class_mapping)
df_tensaoin['class'] = df_tensaoin['valor'].map(class_mapping)

In [ ]:
df_full.describe()

In [ ]:
# @title Testing Time vs Voltage
#df.plot(kind='scatter', x='Testing Time', y='Voltage', s=32, alpha=.8)
df_full.plot(kind='scatter', x='sequencia', y='tensao', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# @title Testing Time vs Voltage
#df.plot(kind='scatter', x='Testing Time', y='Voltage', s=32, alpha=.8)
df_full.plot(kind='scatter', x='sequencia', y='condutancia', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# prompt: Using df, plot the distributions of Voltage, and StdV, for each Sample Group

# Plot the distributions of Voltage for each Sample Group
plt.figure(figsize=(10, 6))
#sns.boxplot(x='Sample Group', y='Voltage', data=df)
sns.boxplot(x='class', y='tensao', data=df_full)
plt.title('Distribution of Voltage for Each Sample Group')
plt.xticks(rotation=45)
plt.show()

# Plot the distributions of Voltage for each Sample Group
plt.figure(figsize=(10, 6))
#sns.boxplot(x='Sample Group', y='Voltage', data=df)
sns.boxplot(x='class', y='condutancia', data=df_full)
plt.title('Distribution of Voltage for Each Sample Group')
plt.xticks(rotation=45)
plt.show()

# Plot the distributions of StdV for each Sample Group
#plt.figure(figsize=(10, 6))
#sns.boxplot(x='Sample Group', y='StdV', data=df)
#plt.title('Distribution of StdV for Each Sample Group')
#plt.xticks(rotation=45)
#plt.show()


In [ ]:
# prompt: Using df, run a hypothesis t-test, comparing class 0 with 1, 0 with 2, 0 with 3, 1 with 2, 1 with 3, 2 with 3, print the result in text, saying if the distributions are similar or not

# Perform t-tests and print results
def compare_distributions(df, class1, class2):
  group1 = df[df['class'] == class1]['tensao']
  group2 = df[df['class'] == class2]['tensao']
  t_statistic, p_value = stats.ttest_ind(group1, group2, equal_var=False)  # Assuming unequal variances

  if p_value < 0.05:
    print(f"The distributions of class {class1} and class {class2} are significantly different (p-value: {p_value:.3f}).")
  else:
    print(f"The distributions of class {class1} and class {class2} are not significantly different (p-value: {p_value:.3f}).")

# Compare all pairs of classes
compare_distributions(df_full, 0, 1)
compare_distributions(df_full, 0, 2)
compare_distributions(df_full, 0, 3)
compare_distributions(df_full, 1, 2)
compare_distributions(df_full, 1, 3)
compare_distributions(df_full, 2, 3)


In [ ]:
#drop nan values of dataset
df_full = df_full.dropna()
df_corrente = df_corrente.dropna()
df_correntein = df_correntein.dropna()
df_tensao = df_tensao.dropna()
df_tensaoin = df_tensaoin.dropna()

In [ ]:
#checking if the classes are balanced
sns.histplot(df_full,x='class')

In [ ]:
#No balanced samples found.
#Iniciating processing of dataset for balancing samples and prepare the matrix input of the AutoML
v_x = df_tensaoin.drop(columns=['DataHora','cabecote','valor','turno','class'])
v_y = df_tensaoin[['class']]

In [ ]:
sns.histplot(v_y,x='class')

In [ ]:
for classe in v_y['class'].unique():
    print('Quantidade de ',classe)
    print(v_y[ v_y['class'] == classe ].count())
    print('--------------')

In [ ]:
#resampling of classes "Aprovado" and "Falha tensao CA baixa", others classes has few samples.
ign = ['Falha polaridade','Falha tensão CA alta']

v_x = df_tensaoin.drop(columns=['DataHora','cabecote','valor','turno','class']).drop(index=df_tensaoin[ (df_tensaoin['class'] == ign[0]) | (df_tensaoin['class'] == ign[1]) ].index).reset_index(drop=True)
v_y = df_tensaoin[['class']].drop(index=df_tensaoin[ (df_tensaoin['class'] == ign[0]) | (df_tensaoin['class'] == ign[1]) ].index).reset_index(drop=True)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler


In [ ]:
rus = RandomUnderSampler(random_state=42, sampling_strategy = 'all')

In [ ]:
vx_re,vy_re = rus.fit_resample(v_x,v_y)

In [ ]:
sns.histplot(vy_re,x='class')

In [ ]:
%matplotlib widget
v_in = vx_re.copy()
v_in['class'] = vy_re


# First model using Pycaret (AutoML), Only Voltage as input.
from pycaret.classification import *
s = setup(v_in, target = 'class', session_id = 123)#, feature_selection = True)
best = create_model('dt', fold = 3)

In [ ]:
evaluate_model(best)

In [ ]:
compare = compare_models()

In [ ]:
s = setup(v_in, target = 'class', session_id = 123)#, feature_selection = True)
best = create_model('et', fold = 3)

In [ ]:
evaluate_model(best)

In [ ]:
df_vcin = df_tensaoin.merge(df_correntein, on=['DataHora', 'item','cabecote','valor','turno','class'], suffixes=('_voltage', '_current'))
                     #.merge(df_stdv, on=['Sample', 'Sample Group','class'], suffixes=('', '_stdv')) \
                     #.merge(df_inivolt, on=['Sample', 'Sample Group','class'], suffixes=('', '_inivolt'))

In [ ]:
#resampling of classes "Aprovado" and "Falha tensao CA baixa", others classes has few samples.
ign = ['Falha polaridade','Falha tensão CA alta']

vc_x = df_vcin.drop(columns=['DataHora','cabecote','valor','turno','class']).drop(index=df_vcin[ (df_vcin['class'] == ign[0]) | (df_vcin['class'] == ign[1]) ].index).reset_index(drop=True)
vc_y = df_vcin[['class']].drop(index=df_vcin[ (df_vcin['class'] == ign[0]) | (df_vcin['class'] == ign[1]) ].index).reset_index(drop=True)

In [ ]:
rus = RandomUnderSampler(random_state=42, sampling_strategy = 'all')
vcx_re,vcy_re = rus.fit_resample(vc_x,vc_y)

In [ ]:
sns.histplot(vcy_re,x='class')

In [ ]:
vc_in = vcx_re.copy()
vc_in['class'] = vcy_re

# First model using Pycaret (AutoML), Voltage and Current as input.
from pycaret.classification import *
s2 = setup(vc_in, target = 'class', session_id = 123)#, feature_selection = True)
best2 = create_model('dt', fold = 3)

In [ ]:
evaluate_model(best2)

In [ ]:
compare = compare_models()

In [ ]:
s2 = setup(vc_in, target = 'class', session_id = 123)#, feature_selection = True)
best2 = create_model('et', fold = 3)

In [ ]:
evaluate_model(best2)

In [ ]:
############ Create a model utilizing % initial voltage as input

In [ ]:
df_vci_in = df_vcin.merge(df_full[ df_full['sequencia']==0 ].drop(columns=['sequencia','tensao','corrente','condutancia','NormV','NormC'])
                          , on=['DataHora', 'item','cabecote','valor','turno','class'])#, suffixes=('_voltage', '_current'))
                     #.merge(df_stdv, on=['Sample', 'Sample Group','class'], suffixes=('', '_stdv')) \
                     #.merge(df_inivolt, on=['Sample', 'Sample Group','class'], suffixes=('', '_inivolt'))

In [ ]:
#resampling of classes "Aprovado" and "Falha tensao CA baixa", others classes has few samples.
#Using "% initial Voltage" as a input too
ign = ['Falha polaridade','Falha tensão CA alta']

vci_x = df_vci_in.drop(columns=['DataHora','cabecote','valor','turno','class']).drop(index=df_vcin[ (df_vci_in['class'] == ign[0]) | (df_vci_in['class'] == ign[1]) ].index).reset_index(drop=True)
vci_y = df_vci_in[['class']].drop(index=df_vci_in[ (df_vci_in['class'] == ign[0]) | (df_vci_in['class'] == ign[1]) ].index).reset_index(drop=True)

In [ ]:
rus = RandomUnderSampler(random_state=42, sampling_strategy = 'all')
vcix_re,vciy_re = rus.fit_resample(vci_x,vci_y)

In [ ]:
sns.histplot(vciy_re,x='class')

In [ ]:
vci_in = vcix_re.copy()
vci_in['class'] = vciy_re

# First model using Pycaret (AutoML), Voltage and Current as input.
from pycaret.classification import *
s3 = setup(vci_in, target = 'class', session_id = 123)#, feature_selection = True)
best3 = create_model('dt', fold = 3)

In [ ]:
evaluate_model(best3)

In [ ]:
compare = compare_models()

In [ ]:
s3 = setup(vci_in, target = 'class', session_id = 123)#, feature_selection = True)
best3 = create_model('et', fold = 3)

In [ ]:
evaluate_model(best3)

In [ ]:
####### Check data balancing about type of battery "item"

In [ ]:
%matplotlib inline
ht = sns.histplot(vci_in,x='class',hue='item',multiple='stack',palette='deep')
sns.move_legend(ht,bbox_to_anchor=(1,1.02), loc='upper left')
#plt.title('Count by "PosY"')
plt.ticklabel_format(style='plain', axis='y')
plt.xticks(rotation=45)
plt.show()

In [ ]:
vci_in['item'].unique()

In [ ]:
# modelling for one type of battery, the most one type frequency "M75LD"

In [ ]:
# First model using Pycaret (AutoML), Voltage and Current as input.
from pycaret.classification import *
s4 = setup(vci_in [ vci_in['item']=='M75LD'], target = 'class',ignore_features = ['item'], session_id = 123)#, feature_selection = True)
best4 = create_model('et', fold = 3)

In [ ]:
evaluate_model(best4)

In [ ]:
### creating model using oversampling, voltage+current+$ initial voltage for two types of defects

from imblearn.over_sampling import RandomOverSampler

# creating SMOTE instancy
ros = RandomOverSampler(random_state=42, sampling_strategy = 'all')

#resampling for all classes
vci_xall = df_vci_in.drop(columns=['DataHora','cabecote','valor','turno','class'])
vci_yall = df_vci_in[['class']]

# balanceando os dados
vci_xall_re, vci_yall_re = ros.fit_resample(vci_xall, vci_yall)

In [ ]:
vci_inall = vci_xall_re.copy()
vci_inall['class'] = vci_yall_re

In [ ]:
%matplotlib inline
ht = sns.histplot(vci_inall,x='class',hue='item',multiple='stack',palette='deep')
sns.move_legend(ht,bbox_to_anchor=(1,1.02), loc='upper left')
#plt.title('Count by "PosY"')
plt.ticklabel_format(style='plain', axis='y')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# First model using Pycaret (AutoML), Voltage and Current as input.
from pycaret.classification import *
s5 = setup(vci_inall, target = 'class', session_id = 123)#, feature_selection = True)
best5 = create_model('et', fold = 3)

In [ ]:
evaluate_model(best5)

## Clustering Analysis

In [ ]:
df_vci_in.groupby('item')['class'].count().reset_index(name='count').sort_values('count')

In [ ]:
input1 = df_vci_in[ df_vci_in['item'] == 'MF72LD' ] 

In [ ]:
# @title Testing Time vs Voltage
df_full[df_full['item'] ==  'MF72LD'].plot(kind='scatter', x='sequencia', y='tensao', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# @title Testing Time vs Voltage
df_full[df_full['item'] ==  'MF72LD'].plot(kind='scatter', x='sequencia', y='condutancia', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
from pycaret.clustering import *
clustering1 = setup(input1,ignore_features = ['Class'], session_id = 123)#, feature_selection = True)


In [ ]:
kmeans1 = create_model('kmeans')

In [ ]:
results1 = assign_model(kmeans1)
results1.head()

In [ ]:
plot_model(kmeans1)


In [ ]:
%matplotlib inline
plot_model(kmeans1, feature = 'class', label=True)
print(results1['class'].unique())
print('Class Aprovado = ',results1[results1['class']=='Aprovado']['Cluster'].unique())
print('Class Falha tensão CA baixa = ',results1[results1['class']=='Aprovado']['Cluster'].unique())
print('Class Falha polaridade = ',results1[results1['class']=='Aprovado']['Cluster'].unique())

In [ ]:
plot_model(kmeans1, plot = 'distance')

In [ ]:
plot_model(kmeans1, plot = 'distribution')

In [ ]:
plot_model(kmeans1, plot = 'elbow')

In [ ]:
results1.to_csv('kmeans_model_MF72LD.csv')

In [ ]:
input2 = df_vci_in[ df_vci_in['item'] == 'M75LD' ] 

In [ ]:
# @title Testing Time vs Voltage
df_full[df_full['item'] ==  'M75LD'].plot(kind='scatter', x='sequencia', y='tensao', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# @title Testing Time vs Voltage
df_full[df_full['item'] ==  'M75LD'].plot(kind='scatter', x='sequencia', y='condutancia', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
from pycaret.clustering import *
clustering2 = setup(input2,ignore_features = ['Class'], session_id = 123)#, feature_selection = True)


In [ ]:
kmeans2 = create_model('kmeans')

In [ ]:
results2 = assign_model(kmeans2)
results2.head()

In [ ]:
plot_model(kmeans2)


In [ ]:
results2.to_csv('kmeans_model_M75LD.csv')

In [ ]:
%matplotlib inline
plot_model(kmeans2, feature = 'class', label=True)
print(results2['class'].unique())
print('Class Aprovado = ',results2[results2['class']=='Aprovado']['Cluster'].unique())
print('Class Falha tensão CA baixa = ',results2[results2['class']=='Aprovado']['Cluster'].unique())
print('Class Falha polaridade = ',results2[results2['class']=='Aprovado']['Cluster'].unique())

In [ ]:
plot_model(kmeans2, plot = 'distance')

In [ ]:
plot_model(kmeans2, plot = 'distribution')

In [ ]:
plot_model(kmeans2, plot = 'elbow')

In [ ]:
### Preprocessing results...